# 并发模式

Bridgic 主要运行在异步事件循环上，同时通过线程无缝支持 I/O 绑定任务。此设计确保了在多样化工作负载下的高并发性。

## Web 内容分析助手

为了探索 Bridgic 对并发的支持，让我们构建一个 Web 内容分析助手，以总结和介绍给定网页的主要内容。步骤如下：

1. 爬取输入 URL 的相关内容。
2. 总结和介绍主要内容

### 1. 爬取相关内容

以 *[Books to Scrape](http://books.toscrape.com/index.html)* 网站为例，我们得到了该网站上某本书页面的 URL。像这样：

In [ ]:
url="http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html"

该页面如下所示：

<br>
<div style="text-align: center;">
<img src="../../../imgs/books_to-scrape.png" alt="参数传递" width="800" height="600">
</div>
<br>

> 注意：我们使用 [Books to Scrape](http://books.toscrape.com/index.html)，这是一个专门为练习网络爬虫而创建的演示网站，在本教程中进行介绍。请注意，这里编写爬虫的目的**不是**构建一个真实的爬虫，而是提供一个简单且安全的示例，以演示 Bridgic 如何处理同步和异步执行模型。

我们使用 `requests` 来获取给定 URL 的网页内容。使用 `pip install requests` 安装 `requests` 包，并像这样爬取页面：

In [ ]:
import requests

def get_web_content(url):  # will return the web content of the given url
    response = requests.get(url)
    return response.text

### 2. 总结并介绍主要内容

我们创建一个代理，输入一个 URL 并爬取相应的页面，然后让模型总结网页的主要内容。

初始化运行时环境。

In [ ]:
import os

# Set the API base and key.
_api_key = os.environ.get("OPENAI_API_KEY")
_api_base = os.environ.get("OPENAI_API_BASE")
_model_name = os.environ.get("OPENAI_MODEL_NAME")

# Import the necessary modules.
from bridgic.core.automa import GraphAutoma, worker
from bridgic.core.model.types import Message, Role
from bridgic.llms.openai_like import OpenAILikeLlm

llm = OpenAILikeLlm(api_base=_api_base, api_key=_api_key, timeout=30)

让我们编写一个网页内容分析助手。

In [ ]:
class WebContentAnalysisAgent(GraphAutoma):
    @worker(is_start=True)
    def crawl_web_content(self, url: str) -> str:
        response = requests.get(url)
        return response.text

    @worker(dependencies=["crawl_web_content"], is_output=True)
    async def analyze_web_content(self, content: str) -> str:
        response = await llm.achat(
            model=_model_name,
            messages=[
                Message.from_text(text="You are a web content analysis assistant. Your task is to analyze the given web content and summarize the main content.", role=Role.SYSTEM),
                Message.from_text(text=content, role=Role.USER),
            ]
        )
        return response.message.content

现在，让我们使用它来帮助我们分析内容。

In [22]:
web_content_analysis_agent = WebContentAnalysisAgent()

# Input the url of the web page to be analyzed.
url = "http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html"

# Call the agent to analyze the web content.
res = await web_content_analysis_agent.arun(url)

# Print the result.
print(f'- - - - - result - - - - -')
print(res)
print(f'- - - - - end - - - - -')


- - - - - result - - - - -
The provided HTML content is from a product page on **Books to Scrape**, a demo website designed for web scraping education. Here's a clear summary of the main content:

---

### **Main Content Summary: A Light in the Attic**

- **Product Title**: *A Light in the Attic*  
- **Author**: Shel Silverstein  
- **Category**: Poetry  
- **Product Type**: Book (Poetry with illustrations)  
- **Price**: £51.77 (excl. and incl. tax; tax is £0.00)  
- **Availability**: In stock (22 units available)  
- **Rating**: 5 stars (all full stars)  
- **Number of Reviews**: 0  

---

### **Product Description Highlights**
- Celebrates its 20th anniversary with a special edition.
- Known for humorous, creative, and rhythmic poetry that appeals to both children and adults.
- Features classic verses such as *"Rockabye Baby"*:
  > *"Rockabye baby, in the treetop / Don't you know a treetop / Is no safe place to rock?"*
- Described as a timeless classic that brings joy and laughter t

<div style="text-align: center; margin: 2rem 0;">
<hr style="border: none; border-top: 2px solid #e2e8f0;">
</div>

## 我们学到了什么？

从这个例子可以看出，Bridgic 可以在同一个 automa 中无缝调度异步和同步的 worker。尽管 `crawl_web_content` 执行了一个阻塞的网络请求，Bridgic 会自动将其分派到一个线程中，以确保事件循环保持不被阻塞。同时，`analyze_web_content` 在事件循环中异步运行。有关详细信息，请参阅 [`Worker`](../../../../reference/bridgic-core/bridgic/core/automa/worker/#bridgic.core.automa.worker.Worker)。

通过这种方式，Bridgic 保持了异步优先的设计，但也通过其线程池提供了对 I/O 绑定操作的内置支持，确保在不同类型的工作负载之间顺利执行。